In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,SimpleRNN
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

In [31]:
file_path_1 = 'data/domain1_train.json'
df1 = pd.read_json(file_path_1, lines=True)


file_path_2 = 'data/domain2_train.json'
df2 = pd.read_json(file_path_2, lines=True).drop('model', axis=1)


df_comb = pd.concat([df1, df2],axis=0,ignore_index=True)

df_comb

,text,label
0,"[70, 746, 825, 109, 2083, 0, 2, 0, 0, 0, 9, 0,...",1
1,"[1209, 179, 1952, 4, 4959, 7, 0, 2, 978, 1522,...",1
2,"[287, 3, 3330, 0, 23, 12, 13, 465, 74, 8, 0, 8...",1
3,"[0, 0, 3, 592, 19, 2, 706, 1439, 2575, 7, 2, 0...",1
4,"[9, 2, 110, 12, 42, 32, 44, 361, 9, 3860, 2358...",1
...,...,...
34395,"[175, 1317, 38, 754, 9, 5, 0, 228, 1, 45, 6, 2...",0
34396,"[466, 5, 70, 1242, 6, 3888, 1, 34, 43, 5, 70, ...",0
34397,"[10, 0, 21, 1650, 18, 5, 1335, 1, 208, 5, 997,...",0
34398,"[18, 39, 316, 133, 365, 2019, 1, 27, 10, 5, 61...",0


In [32]:
X = np.array(df_comb['text'])
y = np.array(df_comb['label'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train[3]) 
print(y_train[3])

[0, 1225, 7, 0, 0, 1813, 3042, 38, 24, 3887, 90, 5, 0, 6, 3059, 953, 3, 0, 2, 0, 9, 1080, 7, 3708, 9, 73, 34, 405, 0, 2129, 1]
1


In [33]:
# Data parameters
num_classes = 2

# Hyperparameters
maxlen = 50  #We set the maximum size of each list to 50.
batch_size = 10
n_epochs = 50
learning_rate = 0.00001
hidden_layers = 200

In [34]:
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[3]) 

[   0 1225    7    0    0 1813 3042   38   24 3887   90    5    0    6
 3059  953    3    0    2    0    9 1080    7 3708    9   73   34  405
    0 2129    1    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [35]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print(y_train.shape)
print(y_train[0])

(27520, 2)
[1. 0.]


In [36]:
#(samples, timesteps, features)
#(batch_size, timesteps, input_dim)
X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape)  #(750, 100, 1)

X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))
print(X_test.shape)

(27520, 50, 1)
(6880, 50, 1)


In [37]:
def vanilla_rnn():
    model = Sequential()
    #model.add(SimpleRNN(50, input_shape = (X_train.shape[1], X_train.shape[2],), return_sequences = False))
    model.add(SimpleRNN(hidden_layers, input_shape = (maxlen,1), return_sequences = False))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    model.summary()
    
    adam = optimizers.Adam(lr = learning_rate)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [38]:
model = KerasClassifier(build_fn = vanilla_rnn, epochs = n_epochs, batch_size = batch_size)
model.fit(X_train, y_train)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_3 (SimpleRNN)    (None, 200)               40400     
                                                                 
 dense_3 (Dense)             (None, 2)                 402       
                                                                 
 activation_3 (Activation)   (None, 2)                 0         
                                                                 
Total params: 40,802
Trainable params: 40,802
Non-trainable params: 0
_________________________________________________________________


Epoch 1/50


C:\Users\Andy\AppData\Local\Temp\ipykernel_18292\3661013997.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = vanilla_rnn, epochs = n_epochs, batch_size = batch_size)
c:\Users\Andy\anaconda3\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


2752/2752 [==============================] - 20s 7ms/step - loss: 0.5584 - accuracy: 0.7136
Epoch 2/50
2752/2752 [==============================] - 19s 7ms/step - loss: 0.5436 - accuracy: 0.7309
Epoch 3/50
2752/2752 [==============================] - 18s 6ms/step - loss: 0.5413 - accuracy: 0.7339
Epoch 4/50
2752/2752 [==============================] - 17s 6ms/step - loss: 0.5380 - accuracy: 0.7364
Epoch 5/50
2752/2752 [==============================] - 19s 7ms/step - loss: 0.5333 - accuracy: 0.7410
Epoch 6/50
2752/2752 [==============================] - 19s 7ms/step - loss: 0.5308 - accuracy: 0.7407
Epoch 7/50
2752/2752 [==============================] - 18s 6ms/step - loss: 0.5290 - accuracy: 0.7414
Epoch 8/50
2752/2752 [==============================] - 18s 6ms/step - loss: 0.5274 - accuracy: 0.7414
Epoch 9/50
2752/2752 [==============================] - 17s 6ms/step - loss: 0.5263 - accuracy: 0.7430
Epoch 10/50
2752/2752 [==============================] - 18s 7ms/step - loss: 0.5246

In [39]:

y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

print(accuracy_score(y_pred, y_test_))

215/215 [==============================] - 1s 5ms/step
0.7582848837209303
